# Pythonのロギングの仕組みを理解する

勉強会の資料用Notebookです。


## Loggerを使う

In [ ]:
from logging import getLogger

# Setup logger
logger = getLogger(__name__)

python の Logger はこうやって初期化するのが一般的です。
あとで解説するので `__name__` とかはおまじないだと思ってください。  

早速このloggerを使ってみましょう。

In [ ]:
from logging import getLogger

logger = getLogger(__name__)

def main():
    logger.debug('This is a Debug message')
    logger.info('This is a Info message')
    logger.warning('This is a Warning message')
    logger.error('This is a Error message')
    logger.critical('This is a Critical message')
    
if __name__ == '__main__':
    main()

他の言語にもあるように Python でも `debug` `info` `warning` `error` `critical` の6種類のログレベルが使用できます。
特に難しいことはないですね。

実行結果も見てみましょう。

>This is a Warning message  
>This is a Error message  
>This is a Critical message

あれ`debug`,` info` はどこへ…。

In [ ]:
from logging import getLogger, DEBUG

logger = getLogger(__name__)

# これかな?
logger.setLevel(DEBUG)


def main():
    logger.debug('This is a Debug message')
    logger.info('This is a Info message')
    logger.warning('This is a Warning message')
    logger.error('This is a Error message')
    logger.critical('This is a Critical message')
    
if __name__ == '__main__':
    main()

`logger` には `setLevel` という、どのレベル以上のものを表示するかという設定があります。
これで `info` や `debug` が無視されていたに違いないので、早速 `DEBUG` 以上をセットしてみましょう。

>This is a Warning message  
>This is a Error message  
>This is a Critical message

ぐぬぬ…

In [ ]:
import sys
from logging import getLogger, DEBUG, StreamHandler

logger = getLogger(__name__)
logger.setLevel(DEBUG)

#  これも？
handler = StreamHandler(sys.stderr)
handler.setLevel(DEBUG)
logger.addHandler(handler)


def main():
    logger.debug('This is a Debug message')
    logger.info('This is a Info message')
    logger.warning('This is a Warning message')
    logger.error('This is a Error message')
    logger.critical('This is a Critical message')
    
if __name__ == '__main__':
    main()

Python の Logger には Handler という概念があります。
Handler というのは書き込まれたログをどう扱うかを決めるためのものです。
デフォルトだと標準エラー出力にログが表示されるだけですが、これをカスタマイズすることで、ファイルに書き込むようにしたり、出力のフォーマットや要素を好きなように変更することが出来ます。
この Handler も ログレベルのしきい値を持っているので、こっちのレベルも変更してみましょう。

>This is a Debug message  
>This is a Info message  
>This is a Warning message  
>This is a Error message  
>This is a Critical message  

👏
無事全部のログが出せましたね。

## Logger について

In [ ]:
logger = getLogger(__name__)

とは一体何なのか


まず引数の中身から。

`__name__`

この Python 組み込みの変数には現在のモジュール名が入っています。

```
* hoge/
     * __init__.py
     * fuga.py
     * piyo/
          * __init__.py
          * piyopiyo.py
```

みたいなパッケージ構成の場合、 

* `fuga.py` での `__name__` は `hoge.fuga`
* `piyo/piyopiyo.py` での `__name__` は `hoge.piyo.piyopiyo`

のようになります。

(但し `if __name__ == '__main__':` のイディオムからも分かるように、対象のファイルを直接実行した場合は、モジュール名ではなく `__main__` という固定値が入るようになっています。今回はややこしくなるので、モジュール名が常に入っているものとして進めさせてください。)

つまり getLogger の1文は、 `logger = getLogger('hoge.fuga')`  のようにモジュール名を入れて呼び出している事と等価です。
実はこの引数、文字列であればなんでもよかったりします。ロガーに名前をつけてるだけですね。

### なぜ __name__ で呼び出すか 
では、なぜ慣例的に `logger = getLogger(__name__)` のように呼び出すかというと、2つのメリットがあるからです。

1つ目はシンプルだからという点です。毎回この定型文を挿入してあげるだけですむので、ロガーのために複雑なことをする必要はありません。正しくファイルを分割していれば、それがそのまま適切なロガー名になります。

2つ目の理由は、**ロガーが階層関係を持てる** という点です。ロガーの名前をドットで区切るとそれが階層になります。なので、`hoge.fuga` と `hoge.piyo.piyopiyo` の2つのロガーは共通の `hoge` というロガーを親に持っています。また名前無しで初期化したロガーはルートロガーと呼ばれ、全てのロガーの親となります。

```
* (ルート)
    * hoge
        * fuga
        * piyo
            * piyopiyo
```

上のパッケージ構成でそれぞれが `__name__` で初期化したロガーをもつ場合、こんな感じの階層が出来上がります。

この階層関係、何が嬉しいのかというと Handler で扱う時に非常に便利になります。ルートロガーからは何も出力させないけど、`hoge` 以下からのログは全て標準出力に書き出したいみたいなことが階層関係を持つことで実現できます。

## Handler について


## Formatter について

## スッキリ書く